## Final Modelling:
**In this notebook we will use deep learning techniques to develop our model, This would be an iterative process, so we reach to the best model** 

Lets start by importing all the required libraries:

In [113]:
# Specific neural network models & layer types
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

import matplotlib.pyplot as plt
from tensorflow.keras import layers
import cv2
import os
import PIL
import pathlib
import pandas as pd

Now using `pathlib.Path()` we can define the path to our dataset which is a collection of images

In [114]:
# storing path to data_dir
data_dir = pathlib.Path("../Data/Alzheimer_s Dataset")

Now we can use the method `.glob` on `data_dir` to to fetch all the images in the specified folder. Just for an example lets make a list of all the paths to `MildDemented` MRIs:

In [115]:
# Making a list of mildDemented scans
mild_Demented = list(data_dir.glob("test/MildDemented/*"))

Now to preview the image(scan) we can also use `PIL.image.open()` In our case we will have to make sure that the path taken by this method is a string so for this reason we will wrap our path in the `str` function:

In [116]:
# Displaying image
img = PIL.Image.open(str(mild_Demented[0]))

Moving forward now we will combine the information from all the cells above to create dictionaries with scan paths and their labels. The scans dictionary will simply organise all paths to the scans according to their class names, where as labels dictionary will record a number for each class name, which in our case would be 0,1,2, and 3.

In [117]:
#Making first dictionary for train data
mri_scans_dict_train = {
    "MildDemented":list(data_dir.glob("train/MildDemented/*")),
    "ModerateDemented":list(data_dir.glob("train/ModerateDemented/*")),
    "NonDemented":list(data_dir.glob("train/NonDemented/*")),
    "VeryMildDemented":list(data_dir.glob("train/VeryMildDemented/*"))
}

In [118]:
# Making dictionary for test data
mri_scans_dict_test = {
    "MildDemented":list(data_dir.glob("test/MildDemented/*")),
    "ModerateDemented":list(data_dir.glob("test/ModerateDemented/*")),
    "NonDemented":list(data_dir.glob("test/NonDemented/*")),
    "VeryMildDemented":list(data_dir.glob("test/VeryMildDemented/*"))
}

In [119]:
# Making Dictionary for relevant labels
mri_scan_labels_dict = {
    "MildDemented":0,
    "ModerateDemented":1,
    "NonDemented":2,
    "VeryMildDemented":3
}

Since the scans dictionary contains paths, we can use it with `cv2`, which can help us read the images on specified path using `cv2.imread()`. By reading the image I mean to say that cv2 will convert the image to a numpy array with its pixel values. Lets represent this with an example, also keep in mind that `imread` method will take in string as an arguement so we will convert our path to string using `str`

In [120]:
# reading an image using imread
cv2.imread(str(mri_scans_dict_train["MildDemented"][0]))

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

Now to implement this method on all the images we will have to run a for loop and then append the `imread()` output to a list, which in our case would be `X_train`, since we will be using the train dictionary first. Additionally from the EDA conducted in our last notebook we know that these images should be grayscaled but yet they come with three color channels so in our for loop we will also convert it to a single channel. Lastly for our `y_train` list we will append label to it in each itteration.  

In [121]:
# Using a for loop to initialise xtrain and ytrain

X_train,y_train = [],[]
for scan_name, images in mri_scans_dict_train.items():
    for image in images:
        img = cv2.imread(str(image))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
        X_train.append(img)
        y_train.append(mri_scan_labels_dict[scan_name])



   

For our ease and model compatibility now we will convert our train lists to numpy arrays using `np.array()`:

In [122]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [123]:
#Sanity Check
X_train.shape

(5121, 208, 176)

Now we will repeat the same process for our test dataset, so this time we will use our test dictionary:

In [124]:
X_test,y_test = [],[]
for scan_name, images in mri_scans_dict_test.items():
    for image in images:
        img = cv2.imread(str(image))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
        X_test.append(img)
        y_test.append(mri_scan_labels_dict[scan_name])




In [125]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [126]:
# Sanity check
X_test.shape

(1279, 208, 176)

At this stage our train and test are both ready to be modelled and evaluated, but we also need to make sure that the pixel values are normalised to the range of 0 to 1, and to do this we will divide our `X_train and X_test` by 255 since pixel values in image data typically range from 0 to 255: 

In [127]:
# Converting train and test to float dtype
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Dividing train and test by 255 to normalise
X_train /= 255
X_test /= 255

Now before we start making Convolutional layers we need to reshape our data with one color channel so we can specify that into our input shape:

In [128]:
# reshaping to add a channel
X_train = X_train.reshape(5121, 208, 176,1)

In [129]:
# reshaping to add a channel
X_test = X_test.reshape(X_test.shape[0], 208, 176,1)

now we can set up an `ImageDataGenerator` for data augmentation, which will apply various transformations to the training images to help improve the model's generalization. However, the provided code `datagen.fit(X_train)` assumes that X_train is a NumPy array of image data, which is typically not the case when using flow_from_dataframe. 

In [130]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2
)

# Fit the generator to the training data
datagen.fit(X_train)


Now lets define a Convolutional Neural Network (CNN) using Keras `Sequential` API:The model will be defined with convolutional layers followed by max-pooling layers, and fully connected layers at the end.

In [131]:
model = Sequential([
    layers.Conv2D(16,kernel_size=(3, 3),activation="relu", input_shape = (208, 176,1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32,kernel_size=(3, 3),activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64,kernel_size=(3, 3),activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128,activation="relu"),
    layers.Dense(64,activation="relu"),
    layers.Dense(4,activation="softmax")

])

The Model summary will show a summary of our model, by telling us the number of layers and their types as well as the parameters:

In [132]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 206, 174, 16)      160       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 103, 87, 16)      0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 101, 85, 32)       4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 50, 42, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 48, 40, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 24, 20, 64)      

Now lets compile this model so moving forward we can fit it onto our dataset and then evaluate the performance:

In [136]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=["accuracy"])

Even though 2 epochs are visible, I already ran 10 epochs before this and the results stood in the same range

In [137]:
model.fit(datagen.flow(X_train,y_train),epochs=2,verbose=1)

Epoch 1/2
161/161 [==============================] - 27s 163ms/step - loss: 1.0740 - accuracy: 0.4855
Epoch 2/2
161/161 [==============================] - 27s 166ms/step - loss: 1.0461 - accuracy: 0.4999


In [138]:
model.evaluate(X_test,y_test)

40/40 [==============================] - 2s 49ms/step - loss: 2.6409 - accuracy: 0.5004


[2.640925884246826, 0.5003909468650818]

The output indicates that our model has completed training and achieved a loss of 2.6409 and an accuracy of 0.5004 (50.04%). Looking at the train accuracy of 0.499(49%) we can now say that because of image augmentation our model is not overfitting.

Moving forward we need to improve the accuracy, and to do that we will use transfer-learning:

## Model Using Transfer Learning

Lets start by initialising `X and y` which we will use to initialise `train and validation` in later cells. To do so we will iterate through `mri_scans_dict_train` dictionary to compile two lists: X for image file paths and y for their corresponding labels

In [53]:
X,y = [],[]
for scan_name, images in mri_scans_dict_train.items():
    for image in images:
        X.append(str(image))
        y.append(str(scan_name))


We Will repeat the same process but now for the test data, to initialise `X_test and y_test`

In [91]:
X_test,y_test = [],[]
for scan_name, images in mri_scans_dict_test.items():
    for image in images:
        X_test.append(str(image))
        y_test.append(str(scan_name))


Now we will combine these lists into a dictionary which will be later used to make a dataframe:

In [92]:
combined_dict_test = {
    "Path":X_test,
    "Label": y_test
}

In [93]:
combined_dict = {
    "Path":X,
    "Label": y
}

Now we can use these dictionaries to create 2 dataframes with 2 columns each one for the `Path` and one for `Label`

In [94]:
df = pd.DataFrame(combined_dict)

In [95]:
df.shape

(5121, 2)

In [96]:
df_test = pd.DataFrame(combined_dict_test)

In [97]:
df_test.shape

(1279, 2)

In [98]:
df_test

,Path,Label
0,../Data/Alzheimer_s Dataset/test/MildDemented/...,MildDemented
1,../Data/Alzheimer_s Dataset/test/MildDemented/...,MildDemented
2,../Data/Alzheimer_s Dataset/test/MildDemented/...,MildDemented
3,../Data/Alzheimer_s Dataset/test/MildDemented/...,MildDemented
4,../Data/Alzheimer_s Dataset/test/MildDemented/...,MildDemented
...,...,...
1274,../Data/Alzheimer_s Dataset/test/VeryMildDemen...,VeryMildDemented
1275,../Data/Alzheimer_s Dataset/test/VeryMildDemen...,VeryMildDemented
1276,../Data/Alzheimer_s Dataset/test/VeryMildDemen...,VeryMildDemented
1277,../Data/Alzheimer_s Dataset/test/VeryMildDemen...,VeryMildDemented


In [99]:
df

,Path,Label
0,../Data/Alzheimer_s Dataset/train/MildDemented...,MildDemented
1,../Data/Alzheimer_s Dataset/train/MildDemented...,MildDemented
2,../Data/Alzheimer_s Dataset/train/MildDemented...,MildDemented
3,../Data/Alzheimer_s Dataset/train/MildDemented...,MildDemented
4,../Data/Alzheimer_s Dataset/train/MildDemented...,MildDemented
...,...,...
5116,../Data/Alzheimer_s Dataset/train/VeryMildDeme...,VeryMildDemented
5117,../Data/Alzheimer_s Dataset/train/VeryMildDeme...,VeryMildDemented
5118,../Data/Alzheimer_s Dataset/train/VeryMildDeme...,VeryMildDemented
5119,../Data/Alzheimer_s Dataset/train/VeryMildDeme...,VeryMildDemented


Now we will split our `df` which contains train data to `train_set` and `val_sel`

In [100]:
from sklearn.model_selection import train_test_split


train_set, val_set = train_test_split(df, test_size=0.2, random_state=42)


In [101]:
#Sanity Check
print(train_set.shape, val_set.shape)

(4096, 2) (1025, 2)


In [103]:
from keras.applications.vgg16 import VGG16, preprocess_input

In this part we first prepare two ImageDataGenerator instances for rescaling the pixel values of images. The first generator is used to create a training data generator from the train_set DataFrame, which loads images and their corresponding labels, resizes them to 224x224 pixels, and normalizes the pixel values. The second generator is used to create a validation data generator from the val_set DataFrame, with an additional 20% validation split for further rescaling and augmentation. These generators will feed batches of images and labels to the model during training and validation, ensuring efficient data handling and preprocessing.

In [34]:
batch_size=32
img_size=(224,224)
image_generator = ImageDataGenerator(rescale=1/255., validation_split=0) #shear_range =.25, zoom_range =.2, horizontal_flip = True, rotation_range=20)     
train_data = image_generator.flow_from_dataframe(dataframe= train_set,x_col="Path",y_col="Label",                                                 
                                                 shuffle=False,
                                                 target_size=img_size, 
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

image_generator = ImageDataGenerator(rescale=1/255,validation_split=0.2) 
validation_data= image_generator.flow_from_dataframe(batch_size=batch_size,
dataframe= val_set,x_col="Path", y_col="Label",                                                 shuffle=False,
                                                 target_size=img_size,
                                                 class_mode='categorical')




Found 4096 validated image filenames belonging to 4 classes.
Found 1025 validated image filenames belonging to 4 classes.


Now we create an ImageDataGenerator instance specifically for the test data, which includes rescaling the pixel values of the images:

In [105]:
# Setup ImageDataGenerator for test data
image_generator_test = ImageDataGenerator(rescale=1/255.)

# Load test data
test_data = image_generator_test.flow_from_dataframe(dataframe=df_test, x_col="Path", y_col="Label",
                                                     shuffle=False, target_size=img_size, 
                                                     batch_size=batch_size, class_mode='categorical')


Found 1279 validated image filenames belonging to 4 classes.


Now we construct a transfer learning model using the pre-trained VGG16 architecture. We first load the VGG16 model without its top layers :

In [37]:
vgg16 = VGG16(input_shape=(224, 224, 3), weights="imagenet", include_top=False)


Moving forward we freeze model's weights to utilize its feature extraction capabilities without altering them during training:

In [38]:
for layer in vgg16.layers:
    layer.trainable = False

We then add custom fully connected layers, including a Flatten layer, a Dense layer with 128 units and ReLU activation, and an output Dense layer with 4 units and softmax activation for multi-class classification. We compile the model with the Adam optimizer and categorical cross-entropy loss, and set up early stopping to halt training if the validation loss does not improve for 10 consecutive epochs, restoring the best weights. This process leverages pre-trained knowledge while tailoring the model to our specific classification task.

In [39]:
from keras.models import Model 
epoch=50
x = Flatten()(vgg16.output)
x = Dense(128, activation='relu')(x)
out = Dense(4, activation='softmax')(x)
modelvgg16 = Model(inputs=vgg16.input, outputs=out)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=10,
                                            restore_best_weights=True)
#compiling
modelvgg16.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])
#Summary
modelvgg16.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

Now finally lets fit the model:

In [40]:
hist_vgg16 = modelvgg16.fit(train_data, epochs=epoch, validation_data=validation_data, callbacks=callback)

Epoch 1/50


2024-07-08 10:27:21.161458: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


128/128 [==============================] - 663s 5s/step - loss: 1.1278 - accuracy: 0.5730 - val_loss: 0.6992 - val_accuracy: 0.7112
Epoch 2/50
128/128 [==============================] - 663s 5s/step - loss: 0.6285 - accuracy: 0.7344 - val_loss: 0.6138 - val_accuracy: 0.7522
Epoch 3/50
128/128 [==============================] - 655s 5s/step - loss: 0.4574 - accuracy: 0.8223 - val_loss: 0.5230 - val_accuracy: 0.7678
Epoch 4/50
128/128 [==============================] - 658s 5s/step - loss: 0.3570 - accuracy: 0.8708 - val_loss: 0.3800 - val_accuracy: 0.8634
Epoch 5/50
128/128 [==============================] - 618s 5s/step - loss: 0.3077 - accuracy: 0.8867 - val_loss: 0.7355 - val_accuracy: 0.6722
Epoch 6/50
128/128 [==============================] - 627s 5s/step - loss: 0.2441 - accuracy: 0.9141 - val_loss: 0.2655 - val_accuracy: 0.9102
Epoch 7/50
128/128 [==============================] - 910s 7s/step - loss: 0.1852 - accuracy: 0.9446 - val_loss: 0.2760 - val_accuracy: 0.8985
Epoch 8/50

KeyboardInterrupt: 

Interrupting the model training at this point because it achieved the best accuracy and loss. It ensures that you preserve the model with the best performance metrics observed during training.
So now we can save this model quickly so we don't lose the learning:


In [41]:
modelvgg16.save("model_vgg16_final.h5")

After saving the model we can load it again, in our case we will do it to evaluate the model, specially on our test data:

In [139]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model

# Load the previously saved model
modelvgg16 = load_model("model_vgg16_final.h5")

Lets evaluate our model on `test_data`:

In [71]:
modelvgg16.evaluate(test_data,verbose=1)

40/40 [==============================] - 160s 4s/step - loss: 1.3514 - accuracy: 0.6943


[1.3513635396957397, 0.694292426109314]

The initial evaluation showed a test accuracy of 69.43%, indicating reasonable performance but room for improvement.

Now to do further evaluation lets get the predictions and so we can generate a classification report:

In [107]:
test_data.reset()  # Reset the generator to ensure it starts from the beginning
y_test_true = test_data.classes  # Get true labels
y_test_pred_prob = modelvgg16.predict(test_data, steps=test_data.samples // batch_size + 1)  # Predict probabilities
y_test_pred = np.argmax(y_test_pred_prob, axis=1) 

40/40 [==============================] - 137s 3s/step


Lets use classifiaction report to further evaluate:

In [108]:
from sklearn.metrics import classification_report


report_test = classification_report(y_test_true, y_test_pred, target_names=list(test_data.class_indices.keys()))
print(report_test)

                  precision    recall  f1-score   support

    MildDemented       0.65      0.28      0.40       179
ModerateDemented       1.00      0.58      0.74        12
     NonDemented       0.71      0.89      0.79       640
VeryMildDemented       0.67      0.58      0.62       448

        accuracy                           0.69      1279
       macro avg       0.76      0.58      0.64      1279
    weighted avg       0.69      0.69      0.68      1279



the report shows that the model achieves an overall accuracy of 69%, with varying performance across classes: "MildDemented" has lower recall (0.28) indicating difficulty in identifying all positive instances, while "NonDemented" performs well with high precision (0.71) and recall (0.89). The report highlights strengths in correctly predicting certain classes and identifies areas, particularly in the "MildDemented" category, where the model's performance could be improved.